# 금속 3D 프린팅 결함 검출 및 분류 - 연합학습 데모

이 노트북은 AprilGAN + CNN 파이프라인을 연합학습 프레임워크에서 시연합니다.

## 파이프라인 개요

1. **AprilGAN**: 제로샷 이상 탐지 (학습 불필요)
2. **CNN**: 결함 유형 분류 (연합학습)
3. **연합학습**: 여러 클라이언트가 가중치만 공유하여 협력 학습


## 1. 환경 설정 및 모듈 임포트


In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import torch
import cv2

# 프로젝트 루트 경로 추가
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# 모듈 임포트
from models.aprilgan import AprilGAN
from models.cnn import DefectClassifierCNN, create_cnn_model
from utils.data_loader import load_defect_data
from utils.bbox_utils import extract_bboxes_from_json
from federated.server import FederatedServer
from federated.client import FederatedClient

print("모듈 로드 완료")


## 2. 데이터 준비


In [ ]:
# 데이터 디렉토리 설정
data_dir = Path("../data")

# 데이터 확인
image_files = list(data_dir.glob("*.jpg"))
print(f"총 이미지 수: {len(image_files)}")

if len(image_files) > 0:
    print(f"첫 번째 이미지: {image_files[0]}")
    
    # 샘플 이미지 확인
    img = cv2.imread(str(image_files[0]))
    if img is not None:
        print(f"이미지 크기: {img.shape}")
        
        # JSON 파일 확인
        json_file = image_files[0].with_suffix(".jpg.json")
        if json_file.exists():
            bboxes, defect_types = extract_bboxes_from_json(json_file)
            print(f"바운딩박스 수: {len(bboxes)}")
            print(f"결함 유형: {defect_types}")
            
print("\n💡 Non-IID 분배: 각 클라이언트가 서로 다른 결함 유형 분포를 가지도록 데이터가 분배됩니다.")


## 3. AprilGAN 모델 초기화 (제로샷)


In [ ]:
# AprilGAN 모델 초기화 (제로샷, 학습 불필요)
aprilgan = AprilGAN()

print("AprilGAN 모델 초기화 완료")
print("AprilGAN은 제로샷 모델로 추가 학습 없이 바로 사용 가능합니다")


## 4. AprilGAN 이상 탐지 시연


In [ ]:
# 샘플 이미지로 AprilGAN 테스트
if len(image_files) > 0:
    sample_image_path = image_files[0]
    
    # 이미지 로드
    image_bgr = cv2.imread(str(sample_image_path))
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    
    # AprilGAN으로 이상 탐지
    result = aprilgan.detect(image_rgb)
    
    # 결과 시각화
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 원본 이미지
    axes[0].imshow(image_rgb)
    axes[0].set_title("원본 이미지")
    axes[0].axis('off')
    
    # 이상 마스크
    axes[1].imshow(result['anomaly_mask'], cmap='hot')
    axes[1].set_title(f"이상 영역 마스크 (점수: {result['anomaly_score']:.3f})")
    axes[1].axis('off')
    
    # 원본 + 이상 영역 오버레이
    overlay = image_rgb.copy()
    overlay[result['anomaly_mask'] == 1] = [255, 0, 0]  # 빨간색으로 표시
    blended = cv2.addWeighted(image_rgb, 0.7, overlay, 0.3, 0)
    axes[2].imshow(blended)
    axes[2].set_title(f"검출된 이상 영역 ({len(result['anomaly_regions'])}개)")
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"검출된 이상 영역 수: {len(result['anomaly_regions'])}")
    for i, region in enumerate(result['anomaly_regions']):
        print(f"  영역 {i+1}: ({region['x1']}, {region['y1']}) - ({region['x2']}, {region['y2']})")


## 5. CNN 모델 초기화 및 데이터 로딩


In [ ]:
# CNN 모델 생성 및 Non-IID 데이터 로딩
from utils.client_data_loader import load_client_data

num_clients = 3
non_iid_alpha = 0.5  # Non-IID 정도 (0.1: 매우 편향, 1.0: 보통, 10.0: 균등)

try:
    # 클라이언트별 Non-IID 데이터 로드
    train_loaders, val_loaders, defect_type_to_idx = load_client_data(
        data_dir=data_dir,
        aprilgan_model=aprilgan,
        num_clients=num_clients,
        train_ratio=0.8,
        batch_size=16,
        patch_size=(224, 224),
        non_iid_alpha=non_iid_alpha,
        verbose=True
    )
    
    num_classes = len(defect_type_to_idx)
    print(f"\n결함 유형 수: {num_classes}")
    print(f"클라이언트별 데이터 로더 생성 완료: {len(train_loaders)}개")
    
    # CNN 모델 생성
    cnn_model = create_cnn_model(
        num_classes=num_classes,
        backbone='resnet18',
        pretrained=True
    )
    
    print(f"CNN 모델 생성 완료 (클래스 수: {num_classes})")
    
except Exception as e:
    print(f"데이터 로딩 오류: {e}")
    print("데모를 위해 더미 데이터 사용")
    num_classes = 5  # 기본값
    cnn_model = create_cnn_model(num_classes=num_classes)
    train_loaders = [None] * num_clients
    val_loaders = [None] * num_clients


## 6. 연합학습 서버 시작


In [ ]:
# 연합학습 서버 생성
server = FederatedServer(
    port=5000,
    num_clients=3,
    min_clients=2
)

# 초기 가중치 설정
initial_weights = cnn_model.state_dict()
server.set_initial_weights(initial_weights)

print("연합학습 서버 준비 완료")
print("서버는 별도 스레드에서 실행됩니다")


In [ ]:
# 서버를 백그라운드에서 시작
import threading
import time

server_thread = threading.Thread(
    target=server.start,
    kwargs={'host': 'localhost', 'debug': False},
    daemon=True
)
server_thread.start()

# 서버가 시작될 때까지 대기
time.sleep(2)

print("서버 시작 완료")


## 7. 연합학습 클라이언트 생성 및 학습


In [ ]:
# 여러 클라이언트 생성 (시뮬레이션)
clients = []
num_clients = 3

for client_id in range(num_clients):
    client = FederatedClient(
        client_id=client_id,
        server_url='http://localhost:5000',
        model=create_cnn_model(num_classes=num_classes)
    )
    clients.append(client)
    print(f"클라이언트 {client_id} 생성 완료")


## 8. 연합학습 라운드 실행


In [ ]:
# 연합학습 라운드 실행
num_rounds = 3

for round_num in range(num_rounds):
    print(f"\n{'='*60}")
    print(f"라운드 {round_num + 1}/{num_rounds}")
    print(f"{'='*60}")
    
    # 1. 각 클라이언트가 서버에서 최신 가중치 가져오기
    print("\n[1단계] 클라이언트가 서버에서 가중치 수신")
    for client in clients:
        client.fetch_aggregated_weights(round_num)
    
    # 2. 각 클라이언트가 로컬 데이터로 학습 (Non-IID 데이터 사용)
    print("\n[2단계] 클라이언트 로컬 학습 (Non-IID 데이터)")
    if 'train_loaders' in locals() and train_loaders[0] is not None:
        # 실제 Non-IID 데이터가 있는 경우
        for client in clients:
            client_train_loader = train_loaders[client.client_id]
            stats = client.train_local(client_train_loader, epochs=1, learning_rate=0.001)
            print(f"  클라이언트 {client.client_id}: Loss={stats['loss']:.4f}, "
                  f"Accuracy={stats['accuracy']:.4f}, Samples={stats['samples']}")
    else:
        # 더미 학습 (실제 구현에서는 실제 데이터 사용)
        print("  더미 학습 모드 (실제 데이터 필요)")
    
    # 3. 각 클라이언트가 학습된 가중치를 서버로 전송
    print("\n[3단계] 클라이언트가 가중치를 서버로 전송")
    for client in clients:
        # 실제 데이터 크기 사용
        if 'train_loaders' in locals() and train_loaders[client.client_id] is not None:
            data_size = len(train_loaders[client.client_id].dataset)
        else:
            data_size = 100  # 더미 값
        client.upload_weights(round_num, data_size)
    
    # 4. 서버가 가중치 집계 (자동으로 수행됨)
    print("\n[4단계] 서버가 가중치 집계")
    time.sleep(1)  # 서버 처리 대기
    
    aggregated_weights = server.get_aggregated_weights()
    if aggregated_weights is not None:
        print(f"  가중치 집계 완료 (라운드 {server.current_round})")
    else:
        print("  아직 집계되지 않음 (더 많은 클라이언트 필요)")
    
    print(f"\n라운드 {round_num + 1} 완료")


## 9. 최종 모델 평가


In [ ]:
# 최종 가중치로 모델 업데이트
final_weights = server.get_aggregated_weights()
if final_weights is not None:
    cnn_model.load_state_dict(final_weights)
    print("최종 집계된 가중치로 모델 업데이트 완료")
    
    # 평가 (실제 데이터가 있는 경우)
    if 'val_loaders' in locals() and val_loaders[0] is not None:
        print("\n[최종 평가] 모든 클라이언트의 검증 데이터로 평가")
        cnn_model.eval()
        
        total_correct = 0
        total_samples = 0
        
        # 모든 클라이언트의 검증 데이터로 평가
        for client_id, val_loader in enumerate(val_loaders):
            client_correct = 0
            client_samples = 0
            
            with torch.no_grad():
                for batch in val_loader:
                    images = batch['image']
                    labels = batch['label']
                    
                    outputs = cnn_model(images)
                    _, predicted = torch.max(outputs, 1)
                    
                    batch_size = labels.size(0)
                    client_samples += batch_size
                    client_correct += (predicted == labels).sum().item()
            
            client_accuracy = client_correct / client_samples if client_samples > 0 else 0.0
            print(f"  클라이언트 {client_id}: {client_accuracy:.4f} ({client_correct}/{client_samples})")
            
            total_samples += client_samples
            total_correct += client_correct
        
        overall_accuracy = total_correct / total_samples if total_samples > 0 else 0.0
        print(f"\n전체 모델 정확도: {overall_accuracy:.4f} ({total_correct}/{total_samples})")
    else:
        print("평가를 위해 실제 데이터가 필요합니다")


## 10. 전체 파이프라인 요약

### 파이프라인 흐름:

```
원본 이미지
    ↓
[AprilGAN] 제로샷 이상 탐지 (학습 불필요)
    ↓
이상 영역 마스크/좌표
    ↓
[CNN] 결함 유형 분류 (연합학습)
    ↓
결함 유형 ("Super Elevation", "Crack", etc.)
```

### 연합학습 구조:

1. **AprilGAN**: 제로샷 모델로 모든 클라이언트에서 동일하게 사용
2. **CNN**: 각 클라이언트가 로컬 데이터로 학습
3. **가중치 전송**: CNN 가중치만 서버로 전송 (데이터는 전송 안 함)
4. **서버 집계**: Federated Averaging으로 가중치 평균화
5. **가중치 배포**: 평균화된 가중치를 모든 클라이언트에 배포

### 핵심 원칙:

- ✅ **데이터 프라이버시**: 원본 데이터는 절대 공유하지 않음
- ✅ **가중치만 전송**: 학습된 모델 가중치만 서버로 전송
- ✅ **서버 집계**: Federated Averaging으로 협력 학습
- ✅ **제로샷 활용**: AprilGAN은 추가 학습 없이 바로 사용
